In [1]:
from ngboost.ngboost import NGBoost
from ngboost.learners import default_tree_learner
from ngboost.scores import MLE
from ngboost.distns import Normal

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


X, Y = load_boston(True)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

ngb = NGBoost(Base=default_tree_learner, Dist=Normal, natural_gradient=True,verbose=False)
ngb.fit(X_train, Y_train)
Y_preds = ngb.predict(X_test)
Y_dists = ngb.pred_dist(X_test)

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, Y_test)
print('Test MSE', test_MSE)

#test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(Y_test.flatten()).mean()
print('Test NLL', test_NLL)

Test MSE 12.553030268509042
Test NLL 3.6348692375220844


# 他のGBMとの比較

In [3]:
from ngboost.learners import default_tree_learner
from ngboost.scores import MLE,CRPS #Maximum Likelihood Estimationl,Continuous Ranked Probability Score
from ngboost.distns import Normal
from ngboost.ngboost import NGBoost

import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import numpy as np
np.random.seed(42)

# Split Dataset
X, Y = load_boston(True)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=42)

# NGBoost
ngb = NGBoost(Base=default_tree_learner, Dist=Normal, natural_gradient=True,verbose=False)
ngb.fit(X_train, Y_train)

Y_train_pred_ngb = ngb.predict(X_train)
Y_test_pred_ngb = ngb.predict(X_test)

# NLL
Y_dists = ngb.pred_dist(X_test)
test_NLL = -Y_dists.logpdf(Y_test.flatten()).mean()

# LightGBM
lgb = lgb.LGBMRegressor()
lgb.fit(X_train, Y_train)

Y_train_pred_lgb = lgb.predict(X_train)
Y_test_pred_lgb = lgb.predict(X_test)

print("-----NGBoost-----")
print('Test NLL', test_NLL)
print('MSE train : %.3f, test : %.3f' % (mean_squared_error(Y_train, Y_train_pred_ngb), mean_squared_error(Y_test, Y_test_pred_ngb)) )

print("-----lightGBM-----")
print('MSE train : %.3f, test : %.3f' % (mean_squared_error(Y_train, Y_train_pred_lgb), mean_squared_error(Y_test, Y_test_pred_lgb)) )

-----NGBoost-----
Test NLL 2.8714493152243596
MSE train : 2.342, test : 7.079
-----lightGBM-----
MSE train : 2.283, test : 8.339
